In [1]:
import pandas as pd
import numpy as np
import lightgbm as lgb

from sklearn.model_selection import StratifiedKFold
from sklearn.metrics import accuracy_score, f1_score, auc, roc_curve
from sklearn.linear_model import LogisticRegression

import matplotlib.pyplot as plt

import random

import warnings 
warnings.filterwarnings("ignore") 

In [2]:
def genData(df_tmp,df_health,runID_pos,disease,randomState=0):
    df_pos = df_tmp[ ( df_tmp['SampleID'].isin(runID_pos) ) & ( df_tmp['disease'] == disease )].drop_duplicates()
    df_pos['label'] = 1
    
    random.seed(randomState)
    neg_idx = random.sample(range(len(df_health)), len(df_pos))
    df_neg = df_health.loc[neg_idx]
    df_neg['label'] = 0
    
    df_health_new = df_health.loc[~df_health.index.isin(neg_idx)]
    df_health_new = df_health_new.reset_index(drop=True)
    
    data = pd.concat([df_pos,df_neg])
    data = data.reset_index(drop=True)
    return data,df_health_new



In [3]:
def Eval(true, prob):
    fpr, tpr, thresholds = roc_curve(true, prob, pos_label=1)
    auc_tmp = auc(fpr, tpr)

    prob[prob<0.5] = 0
    prob[prob>=0.5] = 1
    f1_tmp = f1_score(true,prob)
    return auc_tmp, f1_tmp

In [4]:
def genData_test(df_tmp,df_health,runID_pos,disease,randomState=0):
    df_pos = df_tmp[ df_tmp['SampleID'].isin(runID_pos ) 
                    & ( df_tmp['disease'] == disease )].drop_duplicates()    
    df_pos['label'] = 1    
    df_pos = df_pos.reset_index(drop=True)
    
    random.seed(randomState)
    pos_idx = random.sample(range(len(df_pos)), len(df_health))
    
    data_Multiple = pd.concat([df_pos.loc[pos_idx,df_health.columns],df_health])
    data_Multiple = data_Multiple.reset_index(drop=True)   
    
    return data_Multiple

In [5]:
df_disease = pd.read_csv('patients_ID.csv',low_memory=False)
del df_disease['Unnamed: 0']
df_disease.columns = ['SampleID','disease']

In [6]:
df = pd.read_csv('disease_health.genus_new.Abd',sep='\t',low_memory=False)

disease_SampleID = set(df['SampleID']) & set(df_disease['SampleID'])

In [7]:
df_health = pd.read_csv('health.csv')

In [8]:
df_all = pd.concat([df_disease.loc[df_disease['SampleID'].isin(disease_SampleID)],df_health])
df_all = df_all.reset_index(drop=True)

In [9]:
df_new = df_all.merge(df,how='left',on=['SampleID'])

In [10]:
df_new_s = df_new[df_new['Unclassified']<0.05]
df_new_s = df_new_s.reset_index(drop=True)

In [11]:
df_health_s = df_new_s[df_new_s['disease']=='health']
df_health_s = df_health_s.reset_index(drop=True)

In [12]:
df_health_s.shape

(3274, 2010)

In [13]:
column_names = [x for x in df_new_s.columns if x !='SampleID' and x !='disease' and x !='Unclassified']

In [14]:
disease_sta = pd.DataFrame(df_new_s[['SampleID','disease']].groupby('SampleID',as_index=False).agg(set))

In [15]:
disease_name = 'ibs'
another_name = 'autoimmune'

def isDisease(x):
    if disease_name in x and len(x)==1:
        return 'IBS'
    elif another_name in x and len(x)==1:
        return 'Autoimmune'
    elif disease_name in x and another_name in x:
        return 'Multiple'
    else:
        return 'Other'
    
disease_sta['which_disease'] = disease_sta['disease'].apply(lambda x: isDisease(x))

In [16]:
sum( disease_sta['which_disease']=='Multiple')

385

In [17]:
data_Single_Autoimmune, df_health_new = genData(df_new_s,df_health_s,
                                             set(disease_sta.loc[disease_sta['which_disease']=='Autoimmune','SampleID']),
                                             another_name)   

In [18]:
random.seed(2020)

data_Single_Autoimmune_pos = data_Single_Autoimmune[data_Single_Autoimmune['label']==1]
data_Single_Autoimmune_pos = data_Single_Autoimmune_pos.reset_index(drop=True)

pos_idx = random.sample(range(int(len(data_Single_Autoimmune)/2)), 450) 



data_Single_Autoimmune_neg = data_Single_Autoimmune[data_Single_Autoimmune['label']==0]
data_Single_Autoimmune_neg = data_Single_Autoimmune_neg.reset_index(drop=True)

random.seed(2020)

neg_idx = random.sample(range(int(len(data_Single_Autoimmune)/2)), 450)


data_Single_Autoimmune_s = pd.concat([data_Single_Autoimmune_pos.loc[pos_idx],data_Single_Autoimmune_neg.loc[neg_idx]])
data_Single_Autoimmune_s = data_Single_Autoimmune_s.reset_index(drop=True)

In [19]:
data_Single_Autoimmune_s.shape

(900, 2011)

In [20]:
from mvtpy import mvtest

In [21]:
mv = mvtest.mvtest()

In [22]:
res = []
for each in column_names:
    res.append( ( each, mv.test(data_Single_Autoimmune_s[each], data_Single_Autoimmune_s['label'])['Tn'] ))

In [23]:
res_Autoimmune = sorted(res,key=lambda x:x[1],reverse=True)

In [24]:
res_Autoimmune_s = [x[0] for x in res_Autoimmune if x[1]>0.712]

In [25]:
res_Autoimmune[:11]

[('Cor_Eggerthella', 3.77),
 ('Ent_Tatumella', 2.62),
 ('Ent_Cronobacter', 2.55),
 ('Tis_WAL_1855D', 2.45),
 ('Ent_Edwardsiella', 2.39),
 ('Mog_Eubacterium', 2.39),
 ('Pre_Prevotella', 2.18),
 ('Lac_Tyzzerella', 2.01),
 ('Ery_Erysipelatoclostridium', 1.91),
 ('Bif_Bifidobacterium', 1.76),
 ('Ent_Escherichia_Shigella', 1.73)]

In [26]:
kf = StratifiedKFold(n_splits  = 5, shuffle=True, random_state=2020)

clf = lgb.LGBMClassifier(learning_rate=0.02,max_depth=5,n_estimators=1000,random_state=2020,num_leaves=32,
                         n_jobs=2,subsample=0.8,subsample_freq=5,colsample_bytree=0.8)
train = data_Single_Autoimmune_s[column_names+['label']]
std_ = train[column_names].std()
feats = [x for x in train[column_names].columns if x not in list(std_[std_==0].index)]



auc_single_array = np.zeros(5)
f1_single_array = np.zeros(5)

auc_multiple_array = np.zeros(5)
f1_multiple_array = np.zeros(5)

auc_combine_array = np.zeros(5)
f1_combine_array = np.zeros(5)

count = 0


for train_index, test_index in kf.split(train,train['label']):

    
    train_, test_ = train.loc[train_index], train.loc[test_index]
    y_test = test_['label']


    
    clf.fit(train_[feats],train_['label'])
    y_pred = clf.predict_proba(test_[feats])[:,1]    
    
    auc_single_array[count], f1_single_array[count] = Eval(y_test, y_pred)
    
    data_Multiple = genData_test(df_new_s,train_[train_['label']==0],
                                 set(disease_sta.loc[disease_sta['which_disease']=='Multiple','SampleID']) ,disease_name)
    

    
    clf.fit(data_Multiple[feats],data_Multiple['label'])
    y_pred = clf.predict_proba(test_[feats])[:,1]
    
    auc_multiple_array[count], f1_multiple_array[count] = Eval(y_test, y_pred)
    
    
    
    train_s = train_[train_['label']==1]
    train_s = train_s.reset_index(drop=True)
    #data_Multiple.loc[data_Multiple['label']==1,'label'] = 2
    
    data_Multiple_s = data_Multiple[data_Multiple['label']==1]
    data_Multiple_s = data_Multiple_s.reset_index(drop=True)
    
    

    #data_combine = pd.concat([data_Multiple,train_s])
    data_combine = pd.concat([train_s.loc[:int(len(train_s)/2),],
                              data_Multiple_s.loc[:int(len(data_Multiple_s)/2)],
                              data_Multiple[data_Multiple['label']==0]])
    data_combine = data_combine.reset_index(drop=True)    
    
    clf.fit(data_combine[feats],data_combine['label'])
    y_pred = clf.predict_proba(test_[feats])[:,1]
    auc_combine_array[count], f1_combine_array[count] = Eval(y_test,y_pred)    
    
    count = count + 1

In [27]:
res = []
for each in column_names:
    res.append( ( each, mv.test(data_Multiple[each], data_Multiple['label'])['Tn'] ))

In [28]:
res_Multi = sorted(res,key=lambda x:x[1],reverse=True)

In [29]:
res_Multi_s = [x[0] for x in res_Multi if x[1]>0.712]

In [30]:
len(res_Multi_s)

63

In [31]:
res_Multi[:20]

[('Lac_Eisenbergiella', 7.47),
 ('Cor_Eggerthella', 5.29),
 ('Ent_Cronobacter', 4.12),
 ('Pre_Prevotella', 3.87),
 ('Ent_Edwardsiella', 3.81),
 ('Mog_Eubacterium', 3.69),
 ('Lac_Hungatella', 3.62),
 ('Bif_Bifidobacterium', 3.46),
 ('Ery_Erysipelatoclostridium', 3.35),
 ('Tis_WAL_1855D', 3.15),
 ('Ent_Escherichia_Shigella', 2.9),
 ('Pas_Haemophilus', 2.6),
 ('Pep_Romboutsia', 2.43),
 ('Por_Porphyromonas', 2.38),
 ('Ery_Holdemania', 2.36),
 ('Lac_Tyzzerella', 2.19),
 ('Phy_Phyllobacterium', 1.97),
 ('Rum_Oscillospira', 1.93),
 ('Tis_Finegoldia', 1.91),
 ('Str_Lactococcus', 1.85)]

In [32]:
set(res_Multi_s[:10]) & set(res_Autoimmune_s[:10])

{'Bif_Bifidobacterium',
 'Cor_Eggerthella',
 'Ent_Cronobacter',
 'Ent_Edwardsiella',
 'Ery_Erysipelatoclostridium',
 'Mog_Eubacterium',
 'Pre_Prevotella',
 'Tis_WAL_1855D'}

In [33]:
len( set(res_Multi_s[:10]) | set(res_Autoimmune_s[:10]))

12

In [34]:
df_Autoimmune = pd.DataFrame({'genus':[x[0] for x in res_Autoimmune], 'score_s':[x[1] for x in res_Autoimmune]})

In [35]:
df_Autoimmune

,genus,score_s
0,Cor_Eggerthella,3.77
1,Ent_Tatumella,2.62
2,Ent_Cronobacter,2.55
3,Tis_WAL_1855D,2.45
4,Ent_Edwardsiella,2.39
...,...,...
2002,Met_LD28,0.00
2003,Rho_Planktomarina,0.00
2004,Rho_Methyloligella,0.00
2005,Phy_Phycisphaeraceae_Group,0.00


In [36]:
df_Multi = pd.DataFrame({'genus':[x[0] for x in res_Multi], 'score_m':[x[1] for x in res_Multi]})

In [37]:
heatMap = df_Multi.merge(df_Autoimmune)

In [38]:
heatMap_s = heatMap[heatMap['genus'].isin(set(res_Multi_s[:10]) | set(res_Autoimmune_s[:10]))]
heatMap_s = heatMap_s.reset_index(drop=True)

In [39]:
heatMap_s

,genus,score_m,score_s
0,Lac_Eisenbergiella,7.47,1.05
1,Cor_Eggerthella,5.29,3.77
2,Ent_Cronobacter,4.12,2.55
3,Pre_Prevotella,3.87,2.18
4,Ent_Edwardsiella,3.81,2.39
5,Mog_Eubacterium,3.69,2.39
6,Lac_Hungatella,3.62,0.48
7,Bif_Bifidobacterium,3.46,1.76
8,Ery_Erysipelatoclostridium,3.35,1.91
9,Tis_WAL_1855D,3.15,2.45


In [40]:
heatMap_s['score_m'] = heatMap_s['score_m'].apply(lambda x: 1 if x>0.712 else 0)

In [41]:
heatMap_s['score_s'] = heatMap_s['score_s'].apply(lambda x: 1 if x>0.712 else 0)

# Microbial biomarkers obtained by distribution-free independence test was similar between SD and MD

In [42]:
heatMap_s

,genus,score_m,score_s
0,Lac_Eisenbergiella,1,1
1,Cor_Eggerthella,1,1
2,Ent_Cronobacter,1,1
3,Pre_Prevotella,1,1
4,Ent_Edwardsiella,1,1
5,Mog_Eubacterium,1,1
6,Lac_Hungatella,1,0
7,Bif_Bifidobacterium,1,1
8,Ery_Erysipelatoclostridium,1,1
9,Tis_WAL_1855D,1,1


# Quartile

In [43]:
data_Single_Autoimmune_s.shape

(900, 2011)

In [44]:
data_Multiple.shape

(720, 2008)

In [45]:
list(heatMap_s['genus'])

['Lac_Eisenbergiella',
 'Cor_Eggerthella',
 'Ent_Cronobacter',
 'Pre_Prevotella',
 'Ent_Edwardsiella',
 'Mog_Eubacterium',
 'Lac_Hungatella',
 'Bif_Bifidobacterium',
 'Ery_Erysipelatoclostridium',
 'Tis_WAL_1855D',
 'Lac_Tyzzerella',
 'Ent_Tatumella']

In [46]:
data_Single_tmp = data_Single_Autoimmune_s[list(heatMap_s['genus'])+['label']]

In [47]:
data_Single_tmp[data_Single_tmp['label']==1].quantile(0.25)

Lac_Eisenbergiella            0.000000
Cor_Eggerthella               0.000000
Ent_Cronobacter               0.000000
Pre_Prevotella                0.000156
Ent_Edwardsiella              0.000000
Mog_Eubacterium               0.000000
Lac_Hungatella                0.000000
Bif_Bifidobacterium           0.000025
Ery_Erysipelatoclostridium    0.000000
Tis_WAL_1855D                 0.000000
Lac_Tyzzerella                0.000000
Ent_Tatumella                 0.000000
label                         1.000000
Name: 0.25, dtype: float64

In [48]:
data_Single_tmp[data_Single_tmp['label']==1].median()

Lac_Eisenbergiella            0.000000
Cor_Eggerthella               0.000000
Ent_Cronobacter               0.000062
Pre_Prevotella                0.000663
Ent_Edwardsiella              0.000000
Mog_Eubacterium               0.000000
Lac_Hungatella                0.000000
Bif_Bifidobacterium           0.000227
Ery_Erysipelatoclostridium    0.000061
Tis_WAL_1855D                 0.000000
Lac_Tyzzerella                0.000000
Ent_Tatumella                 0.000000
label                         1.000000
dtype: float64

In [49]:
data_Single_tmp[data_Single_tmp['label']==1].quantile(0.75)

Lac_Eisenbergiella            0.000201
Cor_Eggerthella               0.000073
Ent_Cronobacter               0.000275
Pre_Prevotella                0.008296
Ent_Edwardsiella              0.000252
Mog_Eubacterium               0.000146
Lac_Hungatella                0.000025
Bif_Bifidobacterium           0.001565
Ery_Erysipelatoclostridium    0.000575
Tis_WAL_1855D                 0.000092
Lac_Tyzzerella                0.000276
Ent_Tatumella                 0.000195
label                         1.000000
Name: 0.75, dtype: float64

In [50]:
data_Single_tmp[data_Single_tmp['label']==0].quantile(0.25)

Lac_Eisenbergiella            0.000000
Cor_Eggerthella               0.000000
Ent_Cronobacter               0.000000
Pre_Prevotella                0.000315
Ent_Edwardsiella              0.000000
Mog_Eubacterium               0.000000
Lac_Hungatella                0.000000
Bif_Bifidobacterium           0.000086
Ery_Erysipelatoclostridium    0.000000
Tis_WAL_1855D                 0.000000
Lac_Tyzzerella                0.000000
Ent_Tatumella                 0.000000
label                         0.000000
Name: 0.25, dtype: float64

In [51]:
data_Single_tmp[data_Single_tmp['label']==0].median()

Lac_Eisenbergiella            0.000000
Cor_Eggerthella               0.000000
Ent_Cronobacter               0.000000
Pre_Prevotella                0.001787
Ent_Edwardsiella              0.000000
Mog_Eubacterium               0.000000
Lac_Hungatella                0.000000
Bif_Bifidobacterium           0.000588
Ery_Erysipelatoclostridium    0.000000
Tis_WAL_1855D                 0.000000
Lac_Tyzzerella                0.000000
Ent_Tatumella                 0.000000
label                         0.000000
dtype: float64

In [52]:
data_Single_tmp[data_Single_tmp['label']==0].quantile(0.75)

Lac_Eisenbergiella            0.000123
Cor_Eggerthella               0.000000
Ent_Cronobacter               0.000188
Pre_Prevotella                0.023693
Ent_Edwardsiella              0.000159
Mog_Eubacterium               0.000034
Lac_Hungatella                0.000000
Bif_Bifidobacterium           0.002838
Ery_Erysipelatoclostridium    0.000343
Tis_WAL_1855D                 0.000193
Lac_Tyzzerella                0.000094
Ent_Tatumella                 0.000000
label                         0.000000
Name: 0.75, dtype: float64

In [53]:
data_Multiple_tmp = data_Multiple[list(heatMap_s['genus'])+['label']]

In [54]:
data_Multiple_tmp[data_Multiple_tmp['label']==1].quantile(0.25)

Lac_Eisenbergiella            0.000000
Cor_Eggerthella               0.000000
Ent_Cronobacter               0.000000
Pre_Prevotella                0.000104
Ent_Edwardsiella              0.000000
Mog_Eubacterium               0.000000
Lac_Hungatella                0.000000
Bif_Bifidobacterium           0.000016
Ery_Erysipelatoclostridium    0.000000
Tis_WAL_1855D                 0.000000
Lac_Tyzzerella                0.000000
Ent_Tatumella                 0.000000
label                         1.000000
Name: 0.25, dtype: float64

In [55]:
data_Multiple_tmp[data_Multiple_tmp['label']==1].median()

Lac_Eisenbergiella            0.000086
Cor_Eggerthella               0.000000
Ent_Cronobacter               0.000083
Pre_Prevotella                0.000388
Ent_Edwardsiella              0.000032
Mog_Eubacterium               0.000000
Lac_Hungatella                0.000000
Bif_Bifidobacterium           0.000171
Ery_Erysipelatoclostridium    0.000095
Tis_WAL_1855D                 0.000000
Lac_Tyzzerella                0.000000
Ent_Tatumella                 0.000000
label                         1.000000
dtype: float64

In [56]:
data_Multiple_tmp[data_Multiple_tmp['label']==1].quantile(0.75)

Lac_Eisenbergiella            0.000599
Cor_Eggerthella               0.000109
Ent_Cronobacter               0.000290
Pre_Prevotella                0.003091
Ent_Edwardsiella              0.000265
Mog_Eubacterium               0.000178
Lac_Hungatella                0.000123
Bif_Bifidobacterium           0.001148
Ery_Erysipelatoclostridium    0.000822
Tis_WAL_1855D                 0.000045
Lac_Tyzzerella                0.000223
Ent_Tatumella                 0.000196
label                         1.000000
Name: 0.75, dtype: float64

In [57]:
data_Multiple_tmp[data_Multiple_tmp['label']==0].quantile(0.25)

Lac_Eisenbergiella            0.00000
Cor_Eggerthella               0.00000
Ent_Cronobacter               0.00000
Pre_Prevotella                0.00027
Ent_Edwardsiella              0.00000
Mog_Eubacterium               0.00000
Lac_Hungatella                0.00000
Bif_Bifidobacterium           0.00010
Ery_Erysipelatoclostridium    0.00000
Tis_WAL_1855D                 0.00000
Lac_Tyzzerella                0.00000
Ent_Tatumella                 0.00000
label                         0.00000
Name: 0.25, dtype: float64

In [58]:
data_Multiple_tmp[data_Multiple_tmp['label']==0].median()

Lac_Eisenbergiella            0.000000
Cor_Eggerthella               0.000000
Ent_Cronobacter               0.000000
Pre_Prevotella                0.001470
Ent_Edwardsiella              0.000000
Mog_Eubacterium               0.000000
Lac_Hungatella                0.000000
Bif_Bifidobacterium           0.000666
Ery_Erysipelatoclostridium    0.000000
Tis_WAL_1855D                 0.000000
Lac_Tyzzerella                0.000000
Ent_Tatumella                 0.000000
label                         0.000000
dtype: float64

In [59]:
data_Multiple_tmp[data_Multiple_tmp['label']==0].quantile(0.75)

Lac_Eisenbergiella            0.000125
Cor_Eggerthella               0.000000
Ent_Cronobacter               0.000186
Pre_Prevotella                0.021508
Ent_Edwardsiella              0.000128
Mog_Eubacterium               0.000036
Lac_Hungatella                0.000000
Bif_Bifidobacterium           0.003364
Ery_Erysipelatoclostridium    0.000291
Tis_WAL_1855D                 0.000173
Lac_Tyzzerella                0.000076
Ent_Tatumella                 0.000000
label                         0.000000
Name: 0.75, dtype: float64

# First tree of lgb

In [60]:
clf.fit(train_[feats],train_['label'])

LGBMClassifier(colsample_bytree=0.8, learning_rate=0.02, max_depth=5,
               n_estimators=1000, n_jobs=2, num_leaves=32, random_state=2020,
               subsample=0.8, subsample_freq=5)

In [62]:
graph = lgb.create_tree_digraph(clf, tree_index=0, name='Tree_Autoimmune_SD')
graph.render(view=True)

'Tree_Autoimmune_SD.gv.pdf'

In [63]:
data_Multiple = genData_test(df_new_s,train_[train_['label']==0],
                                 set(disease_sta.loc[disease_sta['which_disease']=='Multiple','SampleID']) ,disease_name)
    
clf.fit(data_Multiple[feats],data_Multiple['label'])

LGBMClassifier(colsample_bytree=0.8, learning_rate=0.02, max_depth=5,
               n_estimators=1000, n_jobs=2, num_leaves=32, random_state=2020,
               subsample=0.8, subsample_freq=5)

In [64]:
graph = lgb.create_tree_digraph(clf, tree_index=0, name='Tree_Autoimmune_MD')
graph.render(view=True)

'Tree_Autoimmune_MD.gv.pdf'